# Example Chapter 5: Using APIs to get Stock Data
In this example we use functions that tap into the yfinance library and Ken French's website to get monthly returns on the Vanguard S\&P 500 ETF (VOO) and short-term US t-bills. We package the yfinance API and the function to scrape Ken French's website in the file "bates_boyer_fletcher_functions.py" for easy use.  Once we have the necessary data in dataframes, we use these data to estimate the annualized expected return, volatility, and Sharpe ratio for VOO. 

In [ ]:
# Load in bates_boyer_fletcher_functions.py from the github repository 
!curl -O https://raw.githubusercontent.com/boyerb/Investments/refs/heads/master/functions/bates_boyer_fletcher_functions.py

In [ ]:
# inport necesary libraries
import bates_boyer_fletcher_functions as invf # here we import the functions from the file we just downloaded
import numpy as np
import pandas as pd


In [ ]:
# Get monthly returns for a given ticker using the yfinance API
ticker = ['^RUA', 'F']
dat = invf.get_monthly_returns(ticker, '1990-01-01', '2023-12-31', tbill_return=False)
print(dat.head())

In [ ]:
# Calculate the annualized average return and volatility
A_Avg = dat.mean() * 12
A_Vol = dat.std() * np.sqrt(12)
print('Anualized Average Return')
print(A_Avg)
print()
print('Annualized Volatility')
print(A_Vol)

In [ ]:
# Download data along with return on short-term tbills from Ken French's website
dat = invf.get_monthly_returns(ticker, '1990-01-01', '2023-12-31', tbill_return=True)
dat['exret0'] = dat[ticker[0]] - dat['RF']
dat['exret1'] = dat[ticker[1]] - dat['RF']
print(dat.head())

In [ ]:
# Calcualte and print Sharpe ratios
A_AVG_exret0 = dat['exret0'].mean() * 12
A_Vol_exret0 = dat[ticker[0]].std() * np.sqrt(12)
Sharpe0 = A_AVG_exret0/A_Vol_exret0

A_AVG_exret1 = dat['exret1'].mean() * 12
A_Vol_exret1 = dat[ticker[1]].std() * np.sqrt(12)
Sharpe1 = A_AVG_exret1/A_Vol_exret1

print('Sharpe Ratios')
print(ticker[0], Sharpe0)
print(ticker[1], Sharpe1)



#Sharpe = A_AVG_exret/A_Vol_exret
#print()
#print('Sharpe Ratio: ', Sharpe)